In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [ ]:
#!pip install category_encoders
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model  import LinearRegression
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor 
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
def Features(x, y):
        
        logistic = LogisticRegression(C=0.35, penalty="l1",solver='liblinear', random_state=5).fit(x, y)
        model = SelectFromModel(logistic, prefit=True)
        x_new = model.transform(x)
    
        # Get back the kept features as a DataFrame with dropped columns as all 0s
        selected_features = pd.DataFrame(model.inverse_transform(x_new), 
                                        index=x.index,
                                        columns=x.columns)
       
        # Dropped columns have values of all 0s, keep other columns 
        good_columns = selected_features.columns[selected_features.var() != 0]
        print (good_columns)
        return good_columns
    
    
def Check(train_train_x,train_train_y,List):
 times = KFold(n_splits=5, shuffle=True, random_state=10)
 for a in List:
  rmse= np.sqrt(-cross_val_score(a, np.log1p(train_train_x), np.log1p(train_train_y), scoring="neg_mean_squared_error",cv=times,n_jobs=-1))
  print('The machine for cross_val_score is:',a,'The score is:',rmse.mean())
 return rmse


#index 1 = return average, index 2 = return emsambered stacking.  
def ensemble(L_bestprediction, index,L_portion = []) :
  if index == 1:
        suma = 0
        for a in L_bestprediction:
            suma += a
        average = suma/len(L_bestprediction)
        return average
  elif index == 2:
        c=0
        y=0
        for b in L_bestprediction:
            if y==0:
                d = b[:int(b.size*L_portion[0])]
                y+=1
                continue
            elif y>0:
                sume = 0
                L_portion[c] += sume
                d = np.hstack((d,b[int(b.size*(L_portion[c])):int(b.size*(L_portion[c]+L_portion[c+1]))]))
                print(d.shape)
                c+=1
        return d

    
#exclude any outlier in the numerical part which helps to improve the accuracy of data.
def outlier(x):
    y=np.array(x,dtype=np.int)
    g=np.array(x,dtype=np.int)
    x=pd.DataFrame(x,dtype=np.int)
    Q1 = x.quantile(0.25)
    Q2 = x.quantile(0.5)
    Q3 = x.quantile(0.75)
    IQR = float(Q3 - Q1)
    c=[]
    for i in range(len(x)):
     y[i] = (y[i] < (Q1 - 1.5 * IQR)) | (y[i] > (Q3 + 1.5 * IQR))
     if y[i] == True:
        g[i]= int(Q2)
    g = pd.DataFrame(g,dtype=np.int)
    return g


test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
test2 = test.copy()
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
train2=train.copy()
train.dropna(axis=0,subset=['SalePrice'],inplace=True)
train_y = train.SalePrice   #The target columns without NaN
Train_y= np.log1p(train_y)
train.drop(['Id','Street','Utilities','LowQualFinSF','3SsnPorch','PoolArea','PoolQC'],axis=1,inplace=True)
test.drop(['Id','Street','Utilities','LowQualFinSF','3SsnPorch','PoolArea','PoolQC'],axis=1,inplace=True)
train.drop(['SalePrice'],axis=1,inplace=True)  #The predictive columns


#for category part:
drop_X = train.select_dtypes(include=['object'])
drop_test_X = test.select_dtypes(include=['object'])


#fill the mising value using SimpleImputer:
SI= SimpleImputer(strategy='constant',fill_value='None')
SI_X = pd.DataFrame(SI.fit_transform(drop_X))
SI_Test_X= pd.DataFrame(SI.fit_transform(drop_test_X))
SI_X.columns = drop_X.columns
SI_Test_X.columns = drop_test_X.columns



#for numerical part:
number_X = train.select_dtypes(exclude=['object'])
number_test_X = test.select_dtypes(exclude=['object'])
#fill the mising value using SimpleImputer:
SI_Median = SimpleImputer(strategy='constant',fill_value=0)
SI_Median_X = pd.DataFrame(SI_Median.fit_transform(number_X))
SI_Median_test_X = pd.DataFrame(SI_Median.transform(number_test_X))
SI_Median_X.columns = number_X.columns
SI_Median_test_X.columns = number_test_X.columns

#Exclude Outlier:
'''
for i in SI_Median_X.columns:
    SI_Median_X[i]=pd.DataFrame(outlier(SI_Median_X[i]))
'''
#combine numerical part and categorical part:
union_x=pd.concat([SI_X,SI_Median_X],axis=1)
UNION_TEST_X=pd.concat([SI_Test_X,SI_Median_test_X],axis=1)
LIGHT_TRAIN_X=pd.concat([union_x,train2['SalePrice']],axis=1)


#New columns
LIGHT_TRAIN_X["TotalHouse"] = LIGHT_TRAIN_X["TotalBsmtSF"].astype(int) + LIGHT_TRAIN_X["1stFlrSF"].astype(int) + LIGHT_TRAIN_X["2ndFlrSF"].astype(int)   
LIGHT_TRAIN_X["TotalArea"] = LIGHT_TRAIN_X["TotalBsmtSF"] + LIGHT_TRAIN_X["1stFlrSF"] + LIGHT_TRAIN_X["2ndFlrSF"] + LIGHT_TRAIN_X["GarageArea"]
LIGHT_TRAIN_X["+_TotalHouse_OverallQual"] = LIGHT_TRAIN_X["TotalHouse"] * LIGHT_TRAIN_X["OverallQual"] 
LIGHT_TRAIN_X["+_GrLivArea_OverallQual"] = LIGHT_TRAIN_X["GrLivArea"] * LIGHT_TRAIN_X["OverallQual"] 
LIGHT_TRAIN_X["+_oMSZoning_TotalHouse"] = LIGHT_TRAIN_X["MSZoning"] * LIGHT_TRAIN_X["TotalHouse"].astype(int)
LIGHT_TRAIN_X["+_oMSZoning_OverallQual"] = LIGHT_TRAIN_X["MSZoning"] + LIGHT_TRAIN_X["OverallQual"].astype(str) 
LIGHT_TRAIN_X["+_oMSZoning_YearBuilt"] = LIGHT_TRAIN_X["MSZoning"] + LIGHT_TRAIN_X["YearBuilt"].astype(str) 
LIGHT_TRAIN_X["+_oNeighborhood_TotalHouse"] = LIGHT_TRAIN_X["Neighborhood"] * LIGHT_TRAIN_X["TotalHouse"] 
LIGHT_TRAIN_X["+_oNeighborhood_OverallQual"] = LIGHT_TRAIN_X["Neighborhood"] + LIGHT_TRAIN_X["OverallQual"].astype(str)  
LIGHT_TRAIN_X["+_oNeighborhood_YearBuilt"] = LIGHT_TRAIN_X["Neighborhood"] + LIGHT_TRAIN_X["YearBuilt"].astype(str)  
LIGHT_TRAIN_X["+_BsmtFinSF1_OverallQual"] = LIGHT_TRAIN_X["BsmtFinSF1"] * LIGHT_TRAIN_X["OverallQual"] 
LIGHT_TRAIN_X["-_oFunctional_TotalHouse"] = LIGHT_TRAIN_X["Functional"] * LIGHT_TRAIN_X["TotalHouse"] 
LIGHT_TRAIN_X["-_oFunctional_OverallQual"] = LIGHT_TRAIN_X["Functional"] + LIGHT_TRAIN_X["OverallQual"].astype(str)  
LIGHT_TRAIN_X["-_LotArea_OverallQual"] = LIGHT_TRAIN_X["LotArea"] * LIGHT_TRAIN_X["OverallQual"].astype(int) 
LIGHT_TRAIN_X["-_TotalHouse_LotArea"] = LIGHT_TRAIN_X["TotalHouse"] + LIGHT_TRAIN_X["LotArea"] 
LIGHT_TRAIN_X["-_oCondition1_TotalHouse"] = LIGHT_TRAIN_X["Condition1"] * LIGHT_TRAIN_X["TotalHouse"] 
LIGHT_TRAIN_X["-_oCondition1_OverallQual"] = LIGHT_TRAIN_X["Condition1"] + LIGHT_TRAIN_X["OverallQual"].astype(str)  
LIGHT_TRAIN_X["Bsmt"] = LIGHT_TRAIN_X["BsmtFinSF1"] + LIGHT_TRAIN_X["BsmtFinSF2"] + LIGHT_TRAIN_X["BsmtUnfSF"]  
LIGHT_TRAIN_X["TotalPlace"] = LIGHT_TRAIN_X["TotalBsmtSF"] + LIGHT_TRAIN_X["1stFlrSF"] + LIGHT_TRAIN_X["2ndFlrSF"] + LIGHT_TRAIN_X["GarageArea"] + LIGHT_TRAIN_X["OpenPorchSF"]+LIGHT_TRAIN_X["EnclosedPorch"]+LIGHT_TRAIN_X["ScreenPorch"]
LIGHT_TRAIN_X.drop(columns=["TotalBsmtSF","1stFlrSF","2ndFlrSF","GarageArea","OverallQual","GrLivArea","MSZoning","YearBuilt","BsmtFinSF1","Neighborhood","LotArea","Condition1","BsmtFinSF2","BsmtUnfSF","OpenPorchSF","EnclosedPorch","ScreenPorch"])

UNION_TEST_X["TotalHouse"] = UNION_TEST_X["TotalBsmtSF"].astype(int) + UNION_TEST_X["1stFlrSF"].astype(int) + UNION_TEST_X["2ndFlrSF"].astype(int) 
UNION_TEST_X["TotalArea"] = UNION_TEST_X["TotalBsmtSF"] + UNION_TEST_X["1stFlrSF"] + UNION_TEST_X["2ndFlrSF"] + UNION_TEST_X["GarageArea"]
UNION_TEST_X["+_TotalHouse_OverallQual"] = UNION_TEST_X["TotalHouse"] * UNION_TEST_X["OverallQual"] 
UNION_TEST_X["+_GrLivArea_OverallQual"] = UNION_TEST_X["GrLivArea"] * UNION_TEST_X["OverallQual"] 
UNION_TEST_X["+_oMSZoning_TotalHouse"] = UNION_TEST_X["MSZoning"] * UNION_TEST_X["TotalHouse"].astype(int) 
UNION_TEST_X["+_oMSZoning_OverallQual"] = UNION_TEST_X["MSZoning"] + UNION_TEST_X["OverallQual"].astype(str) 
UNION_TEST_X["+_oMSZoning_YearBuilt"] = UNION_TEST_X["MSZoning"] + UNION_TEST_X["YearBuilt"].astype(str)  
UNION_TEST_X["+_oNeighborhood_TotalHouse"] = UNION_TEST_X["Neighborhood"] * UNION_TEST_X["TotalHouse"] 
UNION_TEST_X["+_oNeighborhood_OverallQual"] = UNION_TEST_X["Neighborhood"] + UNION_TEST_X["OverallQual"].astype(str)   
UNION_TEST_X["+_oNeighborhood_YearBuilt"] = UNION_TEST_X["Neighborhood"] + UNION_TEST_X["YearBuilt"].astype(str)   
UNION_TEST_X["+_BsmtFinSF1_OverallQual"] = UNION_TEST_X["BsmtFinSF1"] * UNION_TEST_X["OverallQual"] 
UNION_TEST_X["-_oFunctional_TotalHouse"] = UNION_TEST_X["Functional"] * UNION_TEST_X["TotalHouse"] 
UNION_TEST_X["-_oFunctional_OverallQual"] = UNION_TEST_X["Functional"] + UNION_TEST_X["OverallQual"].astype(str)   
UNION_TEST_X["-_LotArea_OverallQual"] = UNION_TEST_X["LotArea"] * UNION_TEST_X["OverallQual"].astype(int) 
UNION_TEST_X["-_TotalHouse_LotArea"] = UNION_TEST_X["TotalHouse"] + UNION_TEST_X["LotArea"] 
UNION_TEST_X["-_oCondition1_TotalHouse"] = UNION_TEST_X["Condition1"] * UNION_TEST_X["TotalHouse"] 
UNION_TEST_X["-_oCondition1_OverallQual"] = UNION_TEST_X["Condition1"] + UNION_TEST_X["OverallQual"].astype(str) 
UNION_TEST_X["Bsmt"] = UNION_TEST_X["BsmtFinSF1"] + UNION_TEST_X["BsmtFinSF2"] + UNION_TEST_X["BsmtUnfSF"] 
UNION_TEST_X["TotalPlace"] = UNION_TEST_X["TotalBsmtSF"] + UNION_TEST_X["1stFlrSF"] + UNION_TEST_X["2ndFlrSF"] + UNION_TEST_X["GarageArea"] + UNION_TEST_X["OpenPorchSF"]+UNION_TEST_X["EnclosedPorch"]+UNION_TEST_X["ScreenPorch"]
UNION_TEST_X.drop(columns=["TotalBsmtSF","1stFlrSF","2ndFlrSF","GarageArea","OverallQual","GrLivArea","MSZoning","YearBuilt","BsmtFinSF1","Neighborhood","LotArea","Condition1","BsmtFinSF2","BsmtUnfSF","OpenPorchSF","EnclosedPorch","ScreenPorch"])
#encode categorical columns with CatEncoder and log the columns that has positive skewness:

Numeric= UNION_TEST_X.select_dtypes(exclude=['object'])
categorical = UNION_TEST_X.select_dtypes(include=['object']).columns
CE = ce.CatBoostEncoder(cols=categorical)
CE.fit(LIGHT_TRAIN_X[categorical], LIGHT_TRAIN_X['SalePrice'])
LIGHT_TRAIN_X[categorical] = CE.transform(LIGHT_TRAIN_X[categorical])
UNION_TEST_X[categorical] = CE.transform(UNION_TEST_X[categorical])
for a in UNION_TEST_X.columns:
    if a in Numeric.columns and abs(LIGHT_TRAIN_X[a].skew(axis=0))>=1:
        LIGHT_TRAIN_X[a]=np.log1p(LIGHT_TRAIN_X[a])
    elif a not in Numeric.columns:
            if abs(LIGHT_TRAIN_X[a].skew(axis=0))>=1:
                LIGHT_TRAIN_X[a]=np.log1p(LIGHT_TRAIN_X[a])
for a in UNION_TEST_X.columns:
    if a in Numeric.columns and abs(UNION_TEST_X[a].skew(axis=0))>=1 :
        UNION_TEST_X[a]=np.log1p(UNION_TEST_X[a])
    elif a not in Numeric.columns:
            if abs(UNION_TEST_X[a].skew(axis=0))>=1:
                UNION_TEST_X[a]=np.log1p(UNION_TEST_X[a])


#Split the datasize
valid_fraction = 0.2741
valid_size = int(len(union_x) * valid_fraction)
train_train = LIGHT_TRAIN_X[:-2 * valid_size]
train_valid = LIGHT_TRAIN_X[-2 * valid_size:]
feature_cols = train_train.columns.drop('SalePrice')
LIGHT_TRAIN_X = train_train[feature_cols]
LIGHT_VALID_X = train_valid[feature_cols]
train_train_y_1 = train_train['SalePrice']
train_valid_y_1 = train_valid['SalePrice']
train_train_y = np.log1p(train_train_y_1)
train_valid_y = np.log1p(train_valid_y_1)
for each in [train_train, train_valid]:
    print(each['SalePrice'].mean())




#Regularization and remain the good columns:
#Object_Columns=Features(LIGHT_TRAIN_X,train_train_y_1)
'''
Object_Columns=['MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition',
       'LotArea', 'BsmtFinSF2', 'BsmtUnfSF', '2ndFlrSF', 'GarageYrBlt',
       'WoodDeckSF', 'MiscVal', '+_TotalHouse_OverallQual',
       '+_GrLivArea_OverallQual', '+_oMSZoning_TotalHouse',
       '+_oMSZoning_OverallQual', '+_oMSZoning_YearBuilt',
       '+_oNeighborhood_TotalHouse', '+_oNeighborhood_OverallQual',
       '+_oNeighborhood_YearBuilt', '+_BsmtFinSF1_OverallQual',
       '-_oFunctional_TotalHouse', '-_oFunctional_OverallQual',
       '-_LotArea_OverallQual', '-_TotalHouse_LotArea',
       '-_oCondition1_TotalHouse', '-_oCondition1_OverallQual', 'TotalPlace']
LIGHT_TRAIN_X = LIGHT_TRAIN_X[Object_Columns]
LIGHT_VALID_X = LIGHT_VALID_X[Object_Columns]
UNION_TEST_X = UNION_TEST_X[Object_Columns] 
'''
#preprocessing the data to have all datas in the same standard, also known as normalization
scalar = StandardScaler()
standard_x =  pd.DataFrame(data=scalar.fit(LIGHT_TRAIN_X).transform(LIGHT_TRAIN_X), columns=LIGHT_TRAIN_X.columns[:])
standard_valid_x=pd.DataFrame(data=scalar.fit(LIGHT_VALID_X).transform(LIGHT_VALID_X), columns=LIGHT_VALID_X.columns[:])
standard_test_x = pd.DataFrame(data=scalar.fit(UNION_TEST_X).transform(UNION_TEST_X), columns=UNION_TEST_X.columns[:])

#PCA
pca = PCA(n_components= 0.965)
Remained_train_X = pca.fit_transform(standard_x)
Remained_train_X = pd.DataFrame(np.reshape(Remained_train_X,(660,int(Remained_train_X.size/660))),columns=LIGHT_TRAIN_X.columns[:int(Remained_train_X.size/660)])
Remained_valid_X = pca.transform(standard_valid_x)
Remained_valid_X = pd.DataFrame(np.reshape(Remained_valid_X,(800,int(Remained_valid_X.size/800))),columns=LIGHT_TRAIN_X.columns[:int(Remained_valid_X.size/800)])
Remained_test_X = pca.transform(standard_test_x)
Remained_test_X = pd.DataFrame(data=np.reshape(Remained_test_X,(1459,int(Remained_test_X.size/1459))),columns=LIGHT_TRAIN_X.columns[:int(Remained_test_X.size/1459)])

#Light
prepare_new = LGBMRegressor(metric='mae',num_leaves= 60,objective='regression', learning_rate=0.01, n_estimators=1000)
prepare = LGBMRegressor(metric='mae',num_leaves= 60,objective='regression', learning_rate=0.01, n_estimators=20000)
prepare.fit(LIGHT_TRAIN_X,train_train_y,eval_set=[(LIGHT_VALID_X,train_valid_y)], early_stopping_rounds=100)
bestprediction4 = prepare.predict(UNION_TEST_X)

#XGBOOST
my_model_new = XGBRegressor(n_estimators=1000,objective ='reg:squarederror', learning_rate=0.01)
my_model = XGBRegressor(n_estimators=20000,objective ='reg:squarederror', learning_rate=0.01)
my_model.fit(Remained_train_X, train_train_y,early_stopping_rounds=100,eval_set=[(Remained_valid_X, train_valid_y)], verbose=False)
bestprediction3=my_model.predict(Remained_test_X)


#Using Linear Regression model:
bestm1=LinearRegression()
bestm1.fit(Remained_train_X,train_train_y)
bestprediction1=bestm1.predict(Remained_test_X)


#RandomTreeRegressor
bestm2 = RandomForestRegressor(random_state=5)
bestm2.fit(Remained_train_X,train_train_y)
bestprediction2=bestm2.predict(Remained_test_X)
#https://towardsdatascience.com/how-to-perform-lasso-and-ridge-regression-in-python-3b3b75541ad8
#Ridge
from sklearn.linear_model import Ridge
ridge=Ridge()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-4,1e-3,1e-2,1,5,10,20]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring="neg_mean_squared_error",cv=5)
ridge_regressor.fit(Remained_train_X,train_train_y)
bestprediction5=ridge_regressor.predict(Remained_test_X)
#Lasso
from sklearn.linear_model import Lasso
lasso=Lasso()
lasso_regressor=GridSearchCV(ridge,parameters,scoring="neg_mean_squared_error",cv=5)
lasso_regressor.fit(Remained_train_X,train_train_y)
bestprediction6=lasso_regressor.predict(Remained_test_X)

List = [prepare_new,my_model_new,bestm1,bestm2,ridge_regressor,lasso_regressor]
#Check(np.expm1(Remained_train_X),train_train_y_1,List)


#Using Stack to combine each portion of estimator's predition together 
List1=[bestprediction5,bestprediction3,bestprediction2,bestprediction6,bestprediction4,bestprediction1]  # List of the model to pass to the function
Portion= [0.2,0.15,0.15,0.5]   #List of proportion for the xList of the model, must add up to 1. for this example, bestprediction4 will have 0.7 size.
c= ensemble(List1,1)
bestprediction=np.expm1(c)

predictionframe = pd.DataFrame({'Id':test2.Id,
                       'SalePrice': bestprediction})
predictionframe.to_csv('submission.csv', index=False)

182036.72424242424
180000.885
[1]	valid_0's l1: 0.295862
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.293382
[3]	valid_0's l1: 0.290818
[4]	valid_0's l1: 0.288338
[5]	valid_0's l1: 0.285821
[6]	valid_0's l1: 0.283357
[7]	valid_0's l1: 0.28095
[8]	valid_0's l1: 0.278528
[9]	valid_0's l1: 0.276122
[10]	valid_0's l1: 0.273819
[11]	valid_0's l1: 0.271472
[12]	valid_0's l1: 0.269159
[13]	valid_0's l1: 0.266894
[14]	valid_0's l1: 0.264701
[15]	valid_0's l1: 0.262447
[16]	valid_0's l1: 0.260222
[17]	valid_0's l1: 0.258028
[18]	valid_0's l1: 0.255845
[19]	valid_0's l1: 0.25375
[20]	valid_0's l1: 0.251624
[21]	valid_0's l1: 0.249586
[22]	valid_0's l1: 0.247507
[23]	valid_0's l1: 0.245501
[24]	valid_0's l1: 0.243463
[25]	valid_0's l1: 0.241489
[26]	valid_0's l1: 0.239499
[27]	valid_0's l1: 0.237567
[28]	valid_0's l1: 0.235679
[29]	valid_0's l1: 0.233778
[30]	valid_0's l1: 0.231839
[31]	valid_0's l1: 0.229976
[32]	valid_0's l1: 0.228142
[33]	valid_0's l1: 0.22

[359]	valid_0's l1: 0.090011
[360]	valid_0's l1: 0.0899766
[361]	valid_0's l1: 0.0899574
[362]	valid_0's l1: 0.0899558
[363]	valid_0's l1: 0.0899273
[364]	valid_0's l1: 0.089908
[365]	valid_0's l1: 0.0898762
[366]	valid_0's l1: 0.0898648
[367]	valid_0's l1: 0.0898513
[368]	valid_0's l1: 0.0898215
[369]	valid_0's l1: 0.0898093
[370]	valid_0's l1: 0.0897901
[371]	valid_0's l1: 0.0897714
[372]	valid_0's l1: 0.0897598
[373]	valid_0's l1: 0.0897266
[374]	valid_0's l1: 0.0896983
[375]	valid_0's l1: 0.0896897
[376]	valid_0's l1: 0.0896802
[377]	valid_0's l1: 0.0896695
[378]	valid_0's l1: 0.0896386
[379]	valid_0's l1: 0.0896135
[380]	valid_0's l1: 0.0896009
[381]	valid_0's l1: 0.0895952
[382]	valid_0's l1: 0.0895686
[383]	valid_0's l1: 0.089557
[384]	valid_0's l1: 0.089547
[385]	valid_0's l1: 0.0895224
[386]	valid_0's l1: 0.0895161
[387]	valid_0's l1: 0.0894992
[388]	valid_0's l1: 0.0894682
[389]	valid_0's l1: 0.0894574
[390]	valid_0's l1: 0.0894373
[391]	valid_0's l1: 0.0894072
[392]	valid_0'

[664]	valid_0's l1: 0.088103
[665]	valid_0's l1: 0.088096
[666]	valid_0's l1: 0.0880995
[667]	valid_0's l1: 0.0881049
[668]	valid_0's l1: 0.0881079
[669]	valid_0's l1: 0.0881021
[670]	valid_0's l1: 0.088098
[671]	valid_0's l1: 0.0881054
[672]	valid_0's l1: 0.088112
[673]	valid_0's l1: 0.0881174
[674]	valid_0's l1: 0.0881289
[675]	valid_0's l1: 0.088125
[676]	valid_0's l1: 0.0881326
[677]	valid_0's l1: 0.0881393
[678]	valid_0's l1: 0.0881454
[679]	valid_0's l1: 0.0881486
[680]	valid_0's l1: 0.0881471
[681]	valid_0's l1: 0.0881539
[682]	valid_0's l1: 0.0881528
[683]	valid_0's l1: 0.0881594
[684]	valid_0's l1: 0.0881634
[685]	valid_0's l1: 0.0881669
[686]	valid_0's l1: 0.0881728
[687]	valid_0's l1: 0.0881759
[688]	valid_0's l1: 0.0881923
[689]	valid_0's l1: 0.0881988
[690]	valid_0's l1: 0.0882022
[691]	valid_0's l1: 0.0881989
[692]	valid_0's l1: 0.0881992
[693]	valid_0's l1: 0.088207
[694]	valid_0's l1: 0.0882145
[695]	valid_0's l1: 0.0882136
[696]	valid_0's l1: 0.0882165
[697]	valid_0's 

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
